## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re, sys, os
sys.path.append("../App")
from download_DB import download_db


In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

##  Cria conexões e baixa banco de dados

In [3]:
directory = download_db(lista_download = ["skate_ug" ,"skate_usinas","skate_leilao"], force_download=False,queries_path="..//App/Queries")
skate_ug = pd.read_parquet(f"{directory}skate_ug.gzip")
skate_usinas = pd.read_parquet(f"{directory}skate_usinas.gzip")
skate_leilao = pd.read_parquet(f"{directory}skate_leilao.gzip")

Carregando arquivo de log...
skate_ug já foi baixado hoje: Dia: 20/12/22 - Horário: 13:09:42.  Portanto não foi baixado novamente.
skate_usinas já foi baixado hoje: Dia: 20/12/22 - Horário: 13:09:46.  Portanto não foi baixado novamente.
skate_leilao já foi baixado hoje: Dia: 20/12/22 - Horário: 13:12:20.  Portanto não foi baixado novamente.


### Importa dataframes que foram salvos

In [4]:
# Carrega dataframe com dados do bdworgs
bdworgs = pd.read_parquet("./Intermediarios/bdworgs.gzip") 

### Tratamento de dados


In [5]:
# Carrega tabela de associações
associassoes = pd.read_excel("./Inputs/associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_R_UG,DatLiberacaoSFGTeste
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpCom_R_UG,DatLiberOpComerRealizado
4,UF,UFUsina
5,NaN,SigTipoGeracao
6,Obras_Real,DatInicioObraRealizado
7,Concretagem_Real,DatConcretagemRealizado
8,Montagem_Real,DatMontagemRealizado
9,Torres_Real,NaN


In [6]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [7]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]

# Colunas provenientes do BD skate_UGs
skate_ug_cols = []

# Colunas provenientes do BD skate_usinas
skate_usinas_col = []

for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        

# Mantém apenas colunas que serão utilizadas        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
skate_leilao = skate_leilao[["IdeUsinaOutorga","DatInicioSuprimento"]]


# Em algum momento as informações sobre torres passaram a ser colocadas em montagem 
mask = bdworgs.Montagem_Real.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]

# Renomeia colunas do bdworgs para o padrão do skate
bdworgs.rename(columns=dict_bdworgs_skate,inplace=True)

# A data operação comercial no bdworgs é na verdade a data de obrigação de operação comercial
# Dat_OC_obrigacao = OC_outorga (Fora do ACR) ou OC_suprimento (ACR) 
bdworgs.rename(columns={"DatUGInicioOpComerOutorgado": "Dat_OC_obrigacao"},inplace=True)

bdworgs.sample(5)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,Num_UG,Pot_UG
1883,29180,2015-12-18,2015-12-15,2015-12-25,PR,2014-01-31,2014-12-31,2015-03-31,nan,ACR,01,6.0
622,31517,2015-08-20,2016-01-01,2015-10-22,RS,2014-11-15,2015-01-15,nan,2015-03-13,ACR,1 a 10,17.9
1068,33569,2017-11-08,2018-11-01,2017-12-23,RN,2017-03-15,nan,2017-10-15,nan,ACR,1 a 11,11.0
807,31793,2014-12-11,2014-10-06,2014-12-16,SP,2013-05-02,nan,nan,nan,Fora do ACR,1,40.0
1137,33683,2017-06-30,2017-12-31,2017-07-26,MA,2016-06-03,2017-01-15,nan,2017-05-15,ACR,4 e 5,4.6


In [8]:
# Ordena dataframe skate_leilao por DatInicioSuprimento
skate_leilao.sort_values(by="DatInicioSuprimento",inplace=True)

# Remove linhas que possuíam datas de leilão para uma mesma usina e mantém apenas aquelas com a primeira data de 
#de DatInicioSuprimento
skate_leilao.drop_duplicates(keep="first",subset= ["IdeUsinaOutorga"],inplace=True)

# Adiciona coluna DatInicioSuprimento ao dataframe skate_usinas
skate_usinas = pd.merge(skate_usinas,
        skate_leilao[["DatInicioSuprimento","IdeUsinaOutorga"]],
        on="IdeUsinaOutorga")


skate_usinas[["DscComercializacaoEnergia","DatInicioSuprimento"]].sample(10)

,DscComercializacaoEnergia,DatInicioSuprimento
2831,Fora do ACR,NaT
272,Fora do ACR,NaT
2274,Fora do ACR,NaT
375,Fora do ACR,NaT
1299,Fora do ACR,NaT
1516,Fora do ACR,NaT
1662,ACR,2024-01-01
761,ACR,2024-01-01
1220,Fora do ACR,NaT
338,ACR,2024-01-01


### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [9]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))


# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

bdworgs.IdeUsinaOutorga = bdworgs.IdeUsinaOutorga.astype(int)
bdworgs.Quant_UG = bdworgs.Quant_UG.astype(int)
bdworgs.Pot_UG = bdworgs.Pot_UG.astype(float)

ceg_df = pd.read_excel("./Inputs/Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,Quant_UG,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,Pot_UG,SigTipoGeracao
1150,33685,2017-07-12,2017-12-31,3,2017-07-26,MA,2016-09-30,2016-10-30,nan,2017-04-15,ACR,6.9,EOL
602,31444,2018-01-05,2018-05-01,1,2018-03-02,PA,2014-09-30,2015-07-11,2016-03-15,nan,ACR,175.0,UHE
179,30597,2012-12-11,2013-07-23,1,2013-01-12,MG,2011-09-12,2012-02-16,2012-05-10,nan,Fora do ACR,4.0,PCH
2263,30202,nan,nan,1,2013-04-12,MA,2012-01-15,nan,nan,nan,ACR,168.8,UTE
1151,33686,2017-07-27,2018-10-01,4,2017-11-07,MA,2016-06-03,2017-01-15,nan,2017-07-06,ACR,9.2,EOL


### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [10]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [11]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [12]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
1.4901161193847656e-08
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [13]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
mask  = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask,"DatUGInicioOpComerOutorgado"] = skate_merged.DatInicioSuprimento
skate_merged.sample(10)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,DatUGInicioOpComerOutorgado,DatLiberOpComerRealizado,Quant_UG,Pot_UG,UFUsina,SigTipoGeracao,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,DscComercializacaoEnergia,DatInicioSuprimento
1776,37878,NaT,2022-03-30,NaT,22.0,55000.00,BA,UFV,NaT,NaT,NaT,Fora do ACR,NaT
3283,49194,NaT,2025-01-01,NaT,29.0,44100.01,MG,UFV,NaT,NaT,NaT,Fora do ACR,NaT
3292,49203,NaT,2025-01-01,NaT,29.0,44100.01,MG,UFV,NaT,NaT,NaT,Fora do ACR,NaT
3223,48767,NaT,2024-09-15,NaT,12.0,39999.96,MG,UFV,NaT,NaT,NaT,Fora do ACR,NaT
4294,50954,NaT,NaT,NaT,NaN,NaN,PI,UFV,NaT,NaT,NaT,Fora do ACR,NaT
58,27929,NaT,2024-10-07,NaT,2.0,8600.00,SP,UTE,2021-11-01,NaT,2021-11-08,Fora do ACR,NaT
3211,48754,NaT,2025-07-31,NaT,14.0,45000.06,PI,UFV,NaT,NaT,NaT,Fora do ACR,NaT
556,32641,2020-08-21,2022-01-01,2020-09-15,4.0,16800.00,BA,EOL,2019-06-01,2019-11-28,2020-06-24,ACR,2022-01-01
2888,47183,NaT,2023-09-05,NaT,2.0,8400.00,BA,EOL,NaT,NaT,NaT,Fora do ACR,NaT
2880,47181,NaT,2023-08-27,NaT,2.0,8400.00,BA,EOL,NaT,NaT,NaT,Fora do ACR,NaT


In [14]:
# Cria colunas Dat_OC_obrigacao em skate_merged
mask_ACR = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask_ACR,"Dat_OC_obrigacao"] = skate_merged.DatInicioSuprimento
skate_merged.loc[~mask_ACR,"Dat_OC_obrigacao"] = skate_merged.DatUGInicioOpComerOutorgado

### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [15]:
colunas_datas = []
for col in set(bdworgs_ug_grouped.columns.to_list()  + skate_merged.columns.to_list()):
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        try:
            skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        except KeyError:
            pass
        try:
            bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")
        except KeyError:
            pass

DatConcretagemRealizado
DatMontagemRealizado
Dat_OC_obrigacao
DatLiberOpComerRealizado
DatLiberacaoSFGTeste
DatUGInicioOpComerOutorgado
DatInicioSuprimento
DatInicioObraRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [16]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatUGInicioOpComerOutorgado
DatInicioSuprimento
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Real


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [17]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.DscComercializacaoEnergia = bd_combined.DscComercializacaoEnergia.astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatLiberacaoSFGTeste,Dat_OC_obrigacao,DatLiberOpComerRealizado,UFUsina,DatInicioObraRealizado,DatConcretagemRealizado,DatMontagemRealizado,Torres_Real,DscComercializacaoEnergia,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatUGInicioOpComerOutorgado,DatInicioSuprimento
1758,31813,2016-05-26,2017-01-01,2016-07-27,CE,2015-07-15,2015-08-10,NaT,2016-01-15,ACR,EOL,7.0,21.00,bdworgs,NaT,NaT
3808,37300,2020-11-26,2024-01-01,2020-12-19,RS,2018-11-30,2020-01-15,2020-04-15,NaN,ACR,PCH,2.0,7900.00,skate,2024-01-01,2024-01-01
843,30172,2013-07-06,2010-06-30,2014-09-04,SC,2010-02-15,2010-07-31,2010-10-01,nan,Fora do ACR,CGH,2.0,2.00,bdworgs,NaT,NaT
2824,32571,NaT,2025-01-01,NaT,SC,2022-10-01,2022-11-22,NaT,NaN,ACR,PCH,1.0,1700.00,skate,2025-01-01,2025-01-01
6382,54627,NaT,2026-01-01,NaT,MG,NaT,NaT,NaT,NaN,Fora do ACR,UFV,33.0,49999.95,skate,2026-01-01,NaT
6510,55914,2022-09-06,2022-05-01,NaT,RJ,2022-04-03,NaT,2022-05-14,NaN,ACR,UTE,1.0,20220.00,skate,2022-05-01,2022-05-01
3501,35271,2022-06-07,2025-01-01,2022-11-04,RN,2021-05-01,2021-05-01,2021-09-22,NaN,ACR,EOL,1.0,3465.00,skate,2025-01-01,2025-01-01
3538,35634,NaT,2024-01-01,NaT,MS,NaT,NaT,NaT,NaN,Fora do ACR,UFV,33.0,49999.95,skate,2024-01-01,NaT
68,2958,2013-07-25,2014-07-01,2013-08-27,SP,NaT,NaT,NaT,nan,ACR,UTE,1.0,4.00,bdworgs,NaT,NaT
2678,32101,2022-07-19,2015-09-01,2022-08-23,BA,2015-06-21,2015-09-10,2016-01-22,NaN,Fora do ACR,EOL,6.0,16200.00,skate,2015-09-01,NaT


In [18]:
bd_combined[["IdeUsinaOutorga","origem","DscComercializacaoEnergia","DatUGInicioOpComerOutorgado","Dat_OC_obrigacao","DatInicioSuprimento"]]

,IdeUsinaOutorga,origem,DscComercializacaoEnergia,DatUGInicioOpComerOutorgado,Dat_OC_obrigacao,DatInicioSuprimento
0,12,bdworgs,ACR,NaT,2007-03-01,NaT
1,12,bdworgs,ACR,NaT,2008-09-01,NaT
2,19,bdworgs,Fora do ACR,NaT,2002-01-01,NaT
3,42,bdworgs,ACR,NaT,2003-11-30,NaT
4,42,bdworgs,ACR,NaT,2005-08-31,NaT
...,...,...,...,...,...,...
6599,50960,skate,Fora do ACR,NaT,NaT,NaT
6600,53724,skate,Fora do ACR,NaT,NaT,NaT
6601,53725,skate,Fora do ACR,NaT,NaT,NaT
6602,53726,skate,Fora do ACR,NaT,NaT,NaT


In [19]:
# Exporta dataframe
bd_combined.to_parquet("./Intermediarios/bd_combined.gzip")